In [2]:
import numpy as np
import pandas as pd
import queue
import math
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
from random import *

from scripts.NN import *
from scripts.io import *

## Part 1: Autoencoder implementation

In [ ]:
# generate 8 bit binary vectors
identity = list(np.identity(8))

In [ ]:
train = [[list(i),list(i)] for i in identity]
train

In [ ]:
nn = NeuralNetwork([[8,3, "sigmoid"], [3,8, "sigmoid"]])

In [ ]:
losses = pd.DataFrame(columns = ['Epoch', 'Train']) 
nn.lr = 0.3
for epoch in range(5000):
    loss = nn.fit(train)
    losses = losses.append({'Epoch' : epoch, 'Train' : loss}, ignore_index=True)

In [ ]:
# Plot loss
fig,ax = plt.subplots()

for name in ['Train']:
    ax.plot(losses['Epoch'],losses[name], label=name)

ax.set_xlabel("epoch")
ax.set_ylabel("loss")
ax.legend(loc='best')

In [ ]:
# pass all 8 vectors in, showing input and the output
# sorry this is so ugly
for i in identity:
    print("input: ",list(i))
    print("output: ",nn.predict(list(i)), "\n")

## Part 2: Adapt for classification, and develop training regime

### 2. a) Describe your process of encoding your training DNA sequences into input vectors in detail. Include a description of how you think the representation might affect your network’s predictions.

For positive training examples, I used all 17 bp each. For the negative examples, I took the last 17 bp of each sequence. Since the neg sequences are the 1000 bp upstream of a yeast gene, I reasoned that the last 17 bp should be closest to the TSS. My hope was that these 17 bp would fall in or around an area that a transcription factor would normally bind. I'm not sure if this logic is correct, but it was a straightforward way to subset the long sequences. I think it is important for both the positive and negative examples to be potential TF binding sites so that the network learns what distinguishes a Rap1 binding site from other binding sites, rather than what distinguishes a transcription factor binding site in general. 

For the encoding, each nucleotide in a DNA sequence is represented as a binary vector of length 4. Example: [1, 0, 0, 0], where a 1 corresponds to which nucleotide it is: [A, C, G, T]. Thus, to get the entire sequence I concatenate these vectors together, resulting in a vector of length 4 * 17. Example: [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]. This one-hot encoding will place equal weight on each nucleotide, rather than place more importance on some nucleotides than others. There may be some relationship between the frequency of one type of nucleotide and Rap1 binding, but I went with one-hot encoding to be safe. 

All of the positive examples were classified as 1, and neg examples classified as 0.

In [5]:
positives, full_negs = read_train_seqs(pos_file="./data/rap1-lieb-positives.txt", neg_file="./data/yeast-upstream-1k-negative.fa")

# just take the last 17 bp of all of the negatives
negatives = [seq[len(seq)-17:len(seq)] for seq in full_negs]
negatives[1:5]

['AAATCTTCACCACCCAA',
 'AGTAAATAACAGATAAT',
 'CATTGTAAAGGAAAACC',
 'AAAATAATAGGTGTAAA']

In [6]:
pos_enc = [[encode_seq(seq),[1]] for seq in positives]


In [7]:
neg_enc = [[encode_seq(seq),[0]] for seq in negatives]

### 3. a) Describe your training regime. How was your training regime designed so as to prevent the negative training data from overwhelming the positive training data?

I first split the data into 80% training, 20% validation. I tried to ensure that the validation set had a good proportion of both positives and negatives. For each epoch, I split the training data into batches, with each batch containing a balanced number of positive and negative examples. Since there are a lot more negative examples than positives, I wanted to make sure the model saw all of the negative examples while still having class-balanced batches. Thus, I re-used some of the positive examples more than once per epoch. To do this, I used two separate queues of training data--one for positive examples and one for negative examples. To create each batch, I dispense a fixed number of positives and negatives from each queue. The epoch ends when the negative queue runs out. The positive queue is replenished with examples when needed.

I also wanted to prevent the model from seeing the same ordering of examples in batches every time. To combat this I repopulated and shuffled the queues after each epoch so that the same positive and negative examples wouldn't always be in the same batch.

In [ ]:
# Put some examples in a validation set. 
# 80/20 split for positives: Since there are only 137 positives, let's save 27 positives.
# Let's just save 64 negs for validation.

pos_test_len = 27
pos_test_idx = sample(range(len(pos_enc)), pos_test_len)
neg_test_len = 64
neg_test_idx = sample(range(len(neg_enc)), neg_test_len)

pos_test = [pos_enc[idx] for idx in pos_test_idx]
neg_test = [neg_enc[idx] for idx in neg_test_idx]

pos_train = [pos_enc[idx] for idx in range(len(pos_enc)) if idx not in pos_test_idx]
neg_train = [neg_enc[idx] for idx in range(len(neg_enc)) if idx not in neg_test_idx]

validation = pos_test + neg_test
shuffle(validation)

In [ ]:
nn = NeuralNetwork([[68,25, "sigmoid"], [25,1, "sigmoid"]])
losses = training(pos_batch_size = 50,neg_batch_size = 155, 
                  n_epochs = 10, nn = nn, neg_train=neg_train, pos_train = pos_train, validation=validation)

### 4. a) Provide an example of the input and output for one true positive sequence and one true negative sequence.

In [ ]:
# positive sequence
print("input: ", pos_enc[0][0])
print("output: ",nn.predict(pos_enc[0][0]))

In [ ]:
# neg sequence
print("input: ", neg_enc[0][0])
print("output: ",nn.predict(neg_enc[0][0]))

### 4. b) Describe your network architecture, and the results of your training. How did your network perform in terms of minimizing error?

I used a network with 68 input nodes, a hidden layer with 25 nodes, and 1 output node. I used all sigmoid activations. My lr was 0.05, and I trained for 10 epochs. Both the training and validation losses steadily decrease.

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots()

for name in ['Train', 'Validation']:
    ax.plot(losses['Epoch'],losses[name], label=name)

ax.set_xlabel("epoch")
ax.set_ylabel("loss")
ax.legend(loc='best')

### 4. c) What was your stop criterion for convergence in your learned parameters? How did you decide this?

If the validation loss goes below, 0.004, the training stops. I chose 0.004 because I ran training for a range of epochs and calculated the AUC on the validation set, and it seems to reach an AUC of 0.99 at that value. If the loss was any lower, the network is not learning much more. Also, I am realizing that I did this wrong and should have probably used a held-out test set that the model has never seen to calculate the AUC.

## Part 3: Cross-validation

### 5. a) How can you use k-fold cross validation to determine your model’s performance?

K-fold cross validation ensures that the network isn't overfitting to a particular training and validation split. It splits the training data into k folds and trains k different networks where the validation set is the k-th fold, and the training set is everything minus that k-th fold. You can then average the AUCs for each fold to determine the model's performance.

### 5. b) Given the size of your dataset, positive and negative examples, how would you select a value for k?

It seems like popular values of k are 3, 5, and 10. I chose 5 so that the network can see a variety of different splits, but also so that the positive examples aren't spread too thin across the folds.

### 5. c) Using the selected value of k, determine a relevant metric of performance for each fold. Describe how your model performed under cross validation.

In [ ]:
k = 5

all_preds, all_actuals = cross_validation(k = k, pos_batch_size=50, neg_batch_size = 211, n_epochs = 1, pos_enc = pos_enc, neg_enc = neg_enc)

In [ ]:
# Get fpr, tpr, and auc for each model
result_table = pd.DataFrame(columns=['k', 'fpr','tpr','auc'])

for i in range(k):  
    fpr, tpr, _ = roc_curve(all_actuals[i],  all_preds[i])
    auc = roc_auc_score(all_actuals[i], all_preds[i])
    
    result_table = result_table.append({'k': i,
                                        'fpr':fpr, 
                                        'tpr':tpr, 
                                        'auc':auc}, ignore_index=True)
result_table

In [ ]:
# ROC 
fig = plt.figure(figsize=(8,6))

for i in result_table.index:
    plt.plot(result_table.loc[i]['fpr'], 
             result_table.loc[i]['tpr'], 
             label="fold={}, AUC={:.3f}".format(i, result_table.loc[i]['auc']))
    
plt.plot([0,1], [0,1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size':13}, loc='lower right')

plt.show()

## Part 4: Extension

### Trying some hyperparameter optimization using grid search

In [8]:
# try a few different architectures, lrs, and change up the positive batch size per iteration
architectures = [[[68,25, "sigmoid"], [25,10, "sigmoid"], [10,1, "sigmoid"]],
                 [[68,10, "sigmoid"], [10,5, "sigmoid"], [5,1, "sigmoid"]],
                 [[68,10, "sigmoid"], [10,1, "sigmoid"]]
                ]
lrs = [0.1, 0.05, 0.01]
pos_batch_size = [30, 50, 80]


In [9]:
k = 5
result_table = pd.DataFrame(columns=['arch', 'lr','pos_batch_size','avg_auc'])

for arch in architectures:
    for lr in lrs:
        for pbs in pos_batch_size:
            nn = NeuralNetwork(setup = arch, lr = lr)
            all_preds, all_actuals = cross_validation(k = k, pos_batch_size=pbs,neg_batch_size = 211, n_epochs = 5, pos_enc = pos_enc, neg_enc = neg_enc)
            aucs = 0
            for i in range(k):  
                auc = roc_auc_score(all_actuals[i], all_preds[i])
                aucs+=auc
            avg_auc = aucs/k
            result_table = result_table.append({'arch': arch,
                                        'lr':lr, 
                                        'pos_batch_size':pbs, 
                                        'avg_auc':avg_auc}, ignore_index=True)

0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4


In [10]:
result_table

,arch,lr,pos_batch_size,avg_auc
0,"[[68, 25, sigmoid], [25, 10, sigmoid], [10, 1,...",0.10,30,0.999861
1,"[[68, 25, sigmoid], [25, 10, sigmoid], [10, 1,...",0.10,50,0.999849
2,"[[68, 25, sigmoid], [25, 10, sigmoid], [10, 1,...",0.10,80,0.999859
3,"[[68, 25, sigmoid], [25, 10, sigmoid], [10, 1,...",0.05,30,0.999944
4,"[[68, 25, sigmoid], [25, 10, sigmoid], [10, 1,...",0.05,50,0.999730
5,"[[68, 25, sigmoid], [25, 10, sigmoid], [10, 1,...",0.05,80,0.999919
6,"[[68, 25, sigmoid], [25, 10, sigmoid], [10, 1,...",0.01,30,0.999885
7,"[[68, 25, sigmoid], [25, 10, sigmoid], [10, 1,...",0.01,50,0.999759
8,"[[68, 25, sigmoid], [25, 10, sigmoid], [10, 1,...",0.01,80,0.999732
9,"[[68, 10, sigmoid], [10, 5, sigmoid], [5, 1, s...",0.10,30,0.999872


In [11]:
#result_table.to_csv("./hyperparameter_tuning_results.csv")

## Part 5: Evaluate your network on the final set.

In [ ]:
# read in test set
with open("./data/rap1-lieb-test.txt") as f:
        test = f.read().splitlines()

In [ ]:
test_enc = [encode_seq(seq) for seq in test]

In [ ]:
pos_test_len = 27
pos_test_idx = sample(range(len(pos_enc)), pos_test_len)
neg_test_len = 64
neg_test_idx = sample(range(len(neg_enc)), neg_test_len)

pos_test = [pos_enc[idx] for idx in pos_test_idx]
neg_test = [neg_enc[idx] for idx in neg_test_idx]

pos_train = [pos_enc[idx] for idx in range(len(pos_enc)) if idx not in pos_test_idx]
neg_train = [neg_enc[idx] for idx in range(len(neg_enc)) if idx not in neg_test_idx]

validation = pos_test + neg_test
shuffle(validation)

In [ ]:
# Going to use optimal hyperparameters from part 4
nn = NeuralNetwork([[68,25, "sigmoid"], [25,1, "sigmoid"]])
losses = training(pos_batch_size = 50,neg_batch_size = 155, n_epochs = 10, nn = nn)

In [ ]:
# test model on test set and print outputs to file
with open('predictions.txt', 'w') as f:
    for t in range(len(test_enc)):
        print(test[t], "\t", nn.predict(test_enc[t][0])[0])